In [6]:
#Imports
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import nltk

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.metrics import confusion_matrix
from sklearn.feature_selection import SelectKBest, chi2

In [7]:
#Data
df = pd.read_csv('prod_data.csv')

# replace number 
df['text'] = [re.sub(r'\d+', 'number',i) for i in df['text']]
df['text'] = [re.sub(r'%', 'percentage',i) for i in df['text']]
df['text'] = [re.sub(r'poly', 'poly poly',i) for i in df['text']]

In [8]:
#Term Frequency-Inverse Document Frequency
tfidf = TfidfVectorizer(lowercase=True, stop_words='english', max_features=1000)
X = tfidf.fit_transform(df['text'])
le = LabelEncoder()
y = le.fit_transform(df['verdict'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=0)

classifier = MultinomialNB()
classifier.fit(X_train, y_train)

y_pred = classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy * 100:.2f}%')

Accuracy: 92.00%


In [9]:
#CountVectorizer
X = df['text']
y = df['verdict']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=0)

vectorizer = CountVectorizer(lowercase=True, ngram_range=(1, 3)) 

X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)


classifier = MultinomialNB()
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy * 100:.2f}%')

Accuracy: 97.00%


In [10]:
#feature selection
vectorizer = CountVectorizer()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=0)
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

k = 2410
selector = SelectKBest(chi2, k=k)
X_train = selector.fit_transform(X_train, y_train)
X_test = selector.transform(X_test)

classifier = MultinomialNB()
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy * 100:.2f}%')

ValueError: k should be <= n_features = 2409; got 2410. Use k='all' to return all features.

In [ ]:
#Bernoulli Naive Bayes 
vectorizer = CountVectorizer(binary=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=0)
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

classifier = BernoulliNB()
classifier.fit(X_train, y_train)

y_pred = classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy * 100:.2f}%')

In [11]:
#testing code

conf_matrix = confusion_matrix(y_test, y_pred)

sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=['Predicted Negative', 'Predicted Positive'], yticklabels=['Actual Negative', 'Actual Positive'])
plt.xlabel('Predicted Label')
plt.ylabel('Actual Label')
plt.title('Confusion Matrix')
plt.show()

# test single product
input_text_vectorized = tfidf.transform(["Details: 100 % calfskin 75 % polyamide and 25 % elastane lining 100 % rubber sole"])
predicted_class_encoded = classifier.predict(input_text_vectorized)

predicted_class_label = le.inverse_transform(predicted_class_encoded)

print(f"Predicted Class: {predicted_class_label[0]}")

NameError: name 'sns' is not defined